In [18]:
from data import dataset, PlantOrgansDataset
from preprocessing import preprocess_image_and_mask
import torchvision.transforms.v2 as T
import torch
import numpy as np
from alexnet import patch_index_to_position, image_to_patches, MyTransform
import train
from train import device

In [19]:
commonTransform = T.Compose([
        T.Resize(size=(2048, 2048)),
        T.ToTensor()
        
        # T.RandomHorizontalFlip(p=0.5),
        # T.RandomVerticalFlip(p=0.5),
        # T.RandomRotation(degrees=45)
    ])
imagesTransform = T.Compose([
    # T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),
    MyTransform(256),
    T.Resize((224, 224)),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
masksTransform = T.Compose([
    T.ToDtype(torch.float16),
    # T.Normalize(mean=[0.0014], std=[0.0031]),
    MyTransform(256),
    # T.Resize((224, 224))
])

c:\ProgramData\anaconda3\Lib\site-packages\torchvision\transforms\v2\_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [20]:
train_validation_data = dataset['train'].train_test_split(test_size=0.2, seed=42)
train_dataset = PlantOrgansDataset(train_validation_data['train'], commonTransform, imagesTransform, masksTransform)
validation_dataset = PlantOrgansDataset(train_validation_data['test'], commonTransform, imagesTransform, masksTransform)
test_dataset = PlantOrgansDataset(dataset['validation'], commonTransform, imagesTransform, masksTransform)


In [21]:
print("train_dataset: ", len(train_dataset))
print("validation_dataset: ", len(validation_dataset))
print("test_dataset: ", len(test_dataset))

train_dataset:  4596
validation_dataset:  1149
test_dataset:  1437


In [22]:
class WrappedDataLoader:
    def __init__(self, loader, func):
        self.loader = loader
        self.func = func

    def __len__(self):
        return len(self.loader)

    def __iter__(self):
        for batch in iter(self.loader):
            batch_cuda = []
            for X, y, size in batch:
                batch_cuda.append(self.func(X, y, size))
            yield batch_cuda

In [23]:
def to_device(X, y, size):
    return X.to(device), y.to(device, dtype=torch.float16), size

In [24]:
batch_size = 64

In [25]:
def custom_collate_fn(batch):
    batchs_amount = len(batch)
    current_images = []
    current_masks = []
    current_sizes = []
    current_length = 0
    for i in range(batchs_amount):
        if current_length == batch_size:
            yield torch.concatenate(current_images), torch.concatenate(current_masks)
        else:    
            images, masks, size = batch[i]
            current_length += len(images)
            current_images.append(images)
            current_sizes.append(size)
            current_masks.append(masks)
    
    # Split images and masks into groups of batch_size
    image_splits = torch.split(images, batch_size, dim=0)  # Split into chunks of batch_size images
    mask_splits = torch.split(masks, batch_size, dim=0)    # Split into chunks of batch_size masks
    
    # Return as a list of batches
    for img_split, mask_split in zip(image_splits, mask_splits):
        yield img_split, mask_split, size



In [31]:
from torch.utils.data import DataLoader
train_loader = WrappedDataLoader(
    DataLoader(train_dataset, batch_size=1, shuffle=False, collate_fn=custom_collate_fn, 
               pin_memory=True, pin_memory_device=[device]), to_device)
valid_loader = WrappedDataLoader(
    DataLoader(validation_dataset, batch_size=1, shuffle=False, collate_fn=custom_collate_fn,
               pin_memory=True, pin_memory_device=[device]), to_device)
test_loader = WrappedDataLoader(
    DataLoader(test_dataset, batch_size=1, shuffle=False, collate_fn=custom_collate_fn,
               pin_memory=True, pin_memory_device=[device]), to_device)

In [27]:
model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False).to(device)

model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=4, bias=True).to(device)

# model = AlexNet([11, 5, 3, 3, 3], [96, 256, 384, 384, 256], [0, 2, 1, 1, 1], [4096, 4096], 4, 224).to(device)

Using cache found in C:\Users\pc/.cache\torch\hub\pytorch_vision_v0.10.0
c:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [28]:
to_image = T.ToPILImage()
mask_to_image = T.Compose([
    T.ToTensor(),
    T.ToDtype(torch.float16),
    T.Normalize(mean=[0.0014], std=[0.0031]),
    T.ToPILImage(),
    
])

In [32]:
for batch in train_loader:
    print(len(batch))
    for X, y, size in batch:
        # print(X.shape)
        predicted = model(X)
        loss_, correct, _ = train.pixel_validate(model, torch.nn.CrossEntropyLoss(), X, y)
        print(loss_, correct)
    break

1
1.3770631551742554 tensor(37.9652, device='cuda:0')


In [33]:
images = []
tensors = []
for batch in train_loader:
    for X, y, size in batch:
        for i in range(len(y)):
            images.append(mask_to_image(y[i]))
            tensors.append(y[i])
        
    break

In [ ]:
torch.concatenate(tensors, dim=1).max()

tensor(0.0157, device='cuda:0', dtype=torch.float16)

In [ ]:
# mask_to_image(train_validation_data['train'][0]['label'])

# mask_to_image(torch.concatenate(tensors, dim=1))

In [34]:
train.evaluate(model, torch.nn.CrossEntropyLoss(), valid_loader)

evaluate  1
evaluate  2
evaluate  3
evaluate  4
evaluate  5
evaluate  6
evaluate  7
evaluate  8
evaluate  9
evaluate  10
evaluate  11
evaluate  12
evaluate  13
evaluate  14
evaluate  15
evaluate  16
evaluate  17
evaluate  18
evaluate  19
evaluate  20
evaluate  21
evaluate  22
evaluate  23
evaluate  24
evaluate  25
evaluate  26
evaluate  27
evaluate  28
evaluate  29
evaluate  30
evaluate  31
evaluate  32
evaluate  33
evaluate  34
evaluate  35
evaluate  36
evaluate  37
evaluate  38
evaluate  39
evaluate  40
evaluate  41
evaluate  42
evaluate  43
evaluate  44
evaluate  45
evaluate  46
evaluate  47
evaluate  48
evaluate  49
evaluate  50
evaluate  51
evaluate  52
evaluate  53
evaluate  54
evaluate  55
evaluate  56
evaluate  57
evaluate  58
evaluate  59
evaluate  60
evaluate  61
evaluate  62
evaluate  63
evaluate  64
evaluate  65
evaluate  66
evaluate  67
evaluate  68
evaluate  69
evaluate  70
evaluate  71
evaluate  72
evaluate  73
evaluate  74
evaluate  75
evaluate  76
evaluate  77
evaluate

KeyboardInterrupt: 

In [ ]:
import os
import torch.optim as optim
import time
from ray import tune
from ray.train import Checkpoint, get_checkpoint, report, RunConfig
from ray.tune.schedulers import ASHAScheduler

In [ ]:
src_path = "C:\\Users\\pc\\Documents\\repos\\mp-2\\nn\\nn-lab2\\"
constants = {
    "criterion": torch.nn.CrossEntropyLoss(),
    "lr": 0.001,
    "n_epochs": 40,
    "saving_model_path": src_path + "models\\raytune"
}
config = {
    "batch_size": tune.grid_search([64]),
    "patch_size": tune.grid_search([256])
}

In [ ]:
def train(config, constants):
    model = torch.hub.load('pytorch/vision:v0.10.0', 'alexnet', pretrained=False).to(device)
    model.classifier[6] = torch.nn.Linear(in_features=4096, out_features=4, bias=True).to(device)

    criterion = constants["criterion"]
    optimizer = optim.Adam(model.parameters(), lr=constants["lr"])
    n_epochs = constants["n_epochs"]
    saving_model_path = os.path.join(constants["saving_model_path"], 
                                "checkpoint_{kernel_size}_{fc_1_out}.model".format(kernel_size=config["kernel_size"], fc_1_out=config["fc_1_out"]))

    train_loader = WrappedDataLoader(DataLoader(train_dataset, batch_size=1, shuffle=True), to_device)
    valid_loader = WrappedDataLoader(DataLoader(validation_dataset, batch_size=1, shuffle=False), to_device)
    test_loader = WrappedDataLoader(DataLoader(test_dataset, batch_size=1, shuffle=False), to_device)

    print('\nFitting nn model')
    start_time = time.time()

    losses_arr = train.fit(n_epochs, model, criterion, optimizer, train_loader, valid_loader)
    print(f'Fit time: {time.time() - start_time} s')

    check_point = torch.load('model.pt', map_location=device)
    model.load_state_dict(check_point)

    test_loss, test_accuracy = train.evaluate(model, criterion, test_loader)

    if saving_model_path is not None:
        print('Saving model')
        torch.save((model.state_dict(), optimizer.state_dict()), saving_model_path)
        checkpoint = Checkpoint.from_directory(constants["saving_model_path"])
        report(
            {"loss": test_loss, "accuracy": test_accuracy},
            checkpoint = checkpoint
        )


In [ ]:
def train_model(num_samples=2, gpus_per_trial=0.125):
    tuner = tune.Tuner(
        tune.with_resources(
            tune.with_parameters(train, constants=constants),
            resources={"cpu": 0.25, "gpu": gpus_per_trial}
        ),
        tune_config=tune.TuneConfig(
            metric="accuracy",
            mode="max",
            # scheduler=scheduler,
            num_samples=num_samples,
            # search_alg=ax_search
        ),
        param_space=config,
        run_config=RunConfig(storage_path=os.path.join(src_path, "raytune"))
    )
    results = tuner.fit()
    
    best_result = results.get_best_result("accuracy", "max")

    return best_result